#### Compute transport of components and write to file (historical and ssp585)

When reading in variables, tp10 can be changed to tp10_BS to obtain historical values shown in components_tpBS. Similarly, tp85 can be changed to tp85_BS to obtain SSP585 values shown in components_tpBS. These BS values are transports corrected for Bering Strait through flow. 

In [2]:
import jCMIP as jc
Clist = jc.readList(('CMIP6listnew'))

from netCDF4 import Dataset
from netCDF4 import num2date
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
import cftime
import gsw
from datetime import date, timedelta
import pandas as pd
from scipy.interpolate import interp1d

#### Historical (overlapping RAPID period)

In [3]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP   = 'historical'

for mod in range (len(totmod)):
    print(('computing file ' + str(mod+1) + ' of ' + str(len(totmod))))

    outfile = ('/home/users/jordi/section26data/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid    = Dataset(outfile,'r')
    lev     = ncid.variables['lev'][:]
    lat     = np.squeeze(ncid.variables['lat'][:,:])
    lon     = np.squeeze(ncid.variables['lon'][:,:])
    tp10    = ncid.variables['tp10'][:,:,:]
    T       = ncid.variables['thetao'][:,:,0,:]
    SP      = ncid.variables['so'][:,:,0,:]
    tau     = ncid.variables['tauuo'][:,0,:]
    dx      = ncid.variables['dx'][0,:]
    ncid.close()
    
    #Ekman calculation
    f = 2*7.2921e-5*np.sin(lat[0]*np.pi/180)
    rho = 1026

    ektp = np.zeros(np.shape(tau))
    ek = np.zeros(np.shape(tau))
    for i in range (len(tau[:,0])):
        for j in range (len(lon)):
            ektp[i,j] = -tau[i,j]/(f*rho)
            ek[i,j]   = ektp[i,j]*dx[j]*1e-6         #v = taux/(rho*f*H), so tp = v*H*L = taux*L/(rho*f)
    
    ektot = np.nansum(ek,axis=1)
    
    time10 = 129

    fs10v = np.zeros(time10)
    AABW10v = np.zeros(time10)
    lNADW10v = np.zeros(time10)
    uNADW10v = np.zeros(time10)
    iw10v = np.zeros(time10)
    tr10v = np.zeros(time10)
    ek10 = ektot[-129::]
    tr10testv = np.zeros(time10)
    ek10testv = np.zeros(time10)   
    #Boundaries vary between years
    for i in range (time10):
        #imax
        tprow10 = np.nansum(tp10[i,:,:],axis=1)
        mocbtv  = -1*np.nancumsum(tprow10[::-1])[::-1]
        ilevmin = np.nanargmin(np.abs(lev-200))         #Find index where lev = 200, so outlier where imax = 0 is solved
        ilevmax = np.nanargmin(np.abs(lev-2000))         #Find index where lev = 2000, so outlier where imax = -1 is solved
        imax2   = np.where(mocbtv[ilevmin:ilevmax] == max(mocbtv[ilevmin:ilevmax]))
        imax1   = imax2[0]
        imax    = imax1[0] + ilevmin                       #+ilevmin since imax[0] = mocbtv[ilev]
        
        imaxtest2   = np.where(mocbtv == max(mocbtv))
        imaxtest1   = imaxtest2[0]
        imaxtest    = imaxtest1[0]
        
        if imax != imaxtest:
            print(str(totmod[mod]) + ' - ' + str(i) + ' - ' + str(lev[imax]))
            
        #Find FS boundary
        tpcol10 = np.nansum(tp10[i,0:imax,:],axis=0)
        mx     = np.nanargmin(np.abs(tpcol10 - max(tpcol10)))
        idx2   = np.where(tpcol10[mx::]<0)                      #Make sure idx > mx (max(tpcol10))
        idx1   = idx2[0]
        idx = idx1[0] + mx
        
        tprow_upmo10v    = np.nansum(tp10[i,:,idx::],axis=1)
        tprow_lowmo10v   = np.nansum(tp10[i,:,:],axis=1)
        
        #PT = 8
        T10  = T[-129::,:,:]
        PTrow = np.nanmean(T10[i,:,:],axis=-1)
        PT8idx = np.nanargmin(np.abs(PTrow-8))
              
        if PT8idx > imax:                                       #Make sure PT8 !> imax
            PT8idx = imax

        #PT = 3
        PT3idx = np.nanargmin(np.abs(PTrow-3))
    
        #Aidx
        for m in range (1,len(tprow_lowmo10v)):                    #start at 1, otherwise the first and last point will be compared
            if tprow_lowmo10v[m]>0 and tprow_lowmo10v[m-1]<0:
                Aidx = m
                
        AABWcheck = tprow_lowmo10v[int(Aidx)::]                    #Make sure sum(AABW) is positive
        if sum(AABWcheck) < 0:
            Aidx = len(tprow_lowmo10v) - 1
            
        if PT3idx > Aidx:                                       #Make sure PT3 !> Aidx
            PT3idx = Aidx 
        
        tprow_fs10v    = np.nansum(tp10[i,0:imax,0:idx],axis=1)
        fs10v[i]       = np.nansum(tprow_fs10v)
        AABW10v[i]     = np.nansum(tprow_lowmo10v[int(Aidx)::])
        lNADW10v[i]    = np.nansum(tprow_lowmo10v[int(PT3idx):int(Aidx)])
        uNADW10v[i]    = np.nansum(tprow_lowmo10v[imax:int(PT3idx)])
        iw10v[i]       = np.nansum(tprow_upmo10v[PT8idx:imax])
        tr10v[i]       = np.nansum(tprow_upmo10v[0:PT8idx]) - ek10[i]         #tr = tp[0-800m] - ek
    
    #Write to file
    outfile = ('/home/users/jordi/components_tp/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid = Dataset(outfile,'w')
    ncid.createDimension('time', time10)
    
    ncid.createVariable('fs10','f8',('time',))
    ncid.createVariable('AABW10','f8',('time',))
    ncid.createVariable('lNADW10','f8',('time',))
    ncid.createVariable('uNADW10','f8',('time',))
    ncid.createVariable('iw10','f8',('time',))
    ncid.createVariable('tr10','f8',('time',))
    ncid.createVariable('ek10','f8',('time',))
    
    ncid.variables['fs10'][:]       = fs10v
    ncid.variables['AABW10'][:]     = AABW10v
    ncid.variables['lNADW10'][:]    = lNADW10v
    ncid.variables['uNADW10'][:]    = uNADW10v
    ncid.variables['iw10'][:]       = iw10v
    ncid.variables['tr10'][:]       = tr10v
    ncid.variables['ek10'][:]       = ek10

    ncid.close()

computing file 1 of 20


/home/users/jmecking001/miniconda3/envs/jordi_2021/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Warning: converting a masked element to nan.


computing file 2 of 20
computing file 3 of 20
computing file 4 of 20
computing file 5 of 20


/home/users/jmecking001/miniconda3/envs/jordi_2021/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: Mean of empty slice


computing file 6 of 20
computing file 7 of 20
computing file 8 of 20
computing file 9 of 20
computing file 10 of 20
computing file 11 of 20
computing file 12 of 20
computing file 13 of 20
computing file 14 of 20
computing file 15 of 20
computing file 16 of 20
computing file 17 of 20
computing file 18 of 20
computing file 19 of 20
computing file 20 of 20


#### Historical - NADW recirculation

In [19]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP   = 'historical'

for mod in range (len(totmod)):
    print(('computing file ' + str(mod+1) + ' of ' + str(len(totmod))))

    outfile = ('/home/users/jordi/section26data/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid    = Dataset(outfile,'r')
    lev     = ncid.variables['lev'][:]
    lat     = np.squeeze(ncid.variables['lat'][:,:])
    lon     = np.squeeze(ncid.variables['lon'][:,:])
    tp10    = ncid.variables['tp10_BS'][:,:,:]
    T       = ncid.variables['thetao'][:,:,0,:]
    SP      = ncid.variables['so'][:,:,0,:]
    tau     = ncid.variables['tauuo'][:,0,:]
    dx      = ncid.variables['dx'][0,:]
    ncid.close()
     
    NADW10N    = np.zeros(time10)
    NADW10S    = np.zeros(time10)    
    #Boundaries vary between years
    for i in range (time10):
        #imax
        tprow10 = np.nansum(tp10[i,:,:],axis=1)
        mocbtv  = -1*np.nancumsum(tprow10[::-1])[::-1]
        ilevmin = np.nanargmin(np.abs(lev-200))         #Find index where lev = 200, so outlier where imax = 0 is solved
        ilevmax = np.nanargmin(np.abs(lev-2000))         #Find index where lev = 2000, so outlier where imax = -1 is solved
        imax2   = np.where(mocbtv[ilevmin:ilevmax] == max(mocbtv[ilevmin:ilevmax]))
        imax1   = imax2[0]
        imax    = imax1[0] + ilevmin                       #+ilevmin since imax[0] = mocbtv[ilev]
        
        imaxtest2   = np.where(mocbtv == max(mocbtv))
        imaxtest1   = imaxtest2[0]
        imaxtest    = imaxtest1[0]
        
        if imax != imaxtest:
            print(str(totmod[mod]) + ' - ' + str(i) + ' - ' + str(lev[imax]))
    
        #Aidx
        for m in range (1,len(tprow_lowmo10v)):                    #start at 1, otherwise the first and last point will be compared
            if tprow_lowmo10v[m]>0 and tprow_lowmo10v[m-1]<0:
                Aidx = m
                
        AABWcheck = tprow_lowmo10v[int(Aidx)::]                    #Make sure sum(AABW) is positive
        if sum(AABWcheck) < 0:
            Aidx = len(tprow_lowmo10v) - 1
        
        #Recirculation boundaries
        NADWcol10  = np.nansum(tp10[i,imax:int(Aidx),:],axis=0)

        mxNADW   = np.nanargmin(NADWcol10)
        idx2NADW = np.where(NADWcol10[mxNADW::]>0)                      #Make sure idx > mx (max(tpcol10))
        idx1NADW = idx2NADW[0]
        idxNADW  = idx1NADW[0] + mxNADW

        NADW10N[i]    = np.nansum(NADWcol10[idxNADW::])
        NADW10S[i]    = np.nansum(NADWcol10[0:idxNADW])
    
    #Write to file
    outfile = ('/home/users/jordi/components_tpBS_recir/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid = Dataset(outfile,'w')
    ncid.createDimension('time', time10)

    ncid.createVariable('NADW10N','f8',('time',))    
    ncid.createVariable('NADW10S','f8',('time',))

    ncid.variables['NADW10N'][:]    = NADW10N
    ncid.variables['NADW10S'][:]    = NADW10S

    ncid.close()

computing file 1 of 19


/home/users/jmecking001/miniconda3/envs/jordi_2021/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Warning: converting a masked element to nan.


computing file 2 of 19
computing file 3 of 19
computing file 4 of 19
computing file 5 of 19


/home/users/jmecking001/miniconda3/envs/jordi_2021/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: Mean of empty slice


computing file 6 of 19
computing file 7 of 19
computing file 8 of 19
computing file 9 of 19
computing file 10 of 19
computing file 11 of 19
computing file 12 of 19
computing file 13 of 19
computing file 14 of 19
computing file 15 of 19
computing file 16 of 19
computing file 17 of 19
computing file 18 of 19
computing file 19 of 19


#### SSP585

In [ ]:
#CNRM-ESM2-1 (r2i1p1f2) has no ssp585
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP   = 'ssp585'

t = 85*12       #Set end time of all models to 2100
for mod in range (len(totmod)):
    print(('computing file ' + str(mod+1) + ' of ' + str(len(totmod))))

    outfile = ('/home/users/jordi/section26data/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid    = Dataset(outfile,'r')
    lev     = ncid.variables['lev'][:]
    lat     = np.squeeze(ncid.variables['lat'][:,:])
    lon     = np.squeeze(ncid.variables['lon'][:,:])
    tp85    = ncid.variables['tp85_BS'][:,:,:]
    T       = ncid.variables['thetao'][0:t,:,0,:]
    SP      = ncid.variables['so'][0:t,:,0,:]
    tau     = ncid.variables['tauuo'][0:t,0,:]
    dx      = ncid.variables['dx'][0,:]
    ncid.close()
    
    #Ekman calculation
    f = 2*7.2921e-5*np.sin(lat[0]*np.pi/180)
    rho = 1026

    ektp = np.zeros(np.shape(tau))
    ek = np.zeros(np.shape(tau))
    for i in range (len(tau[:,0])):
        for j in range (len(lon)):
            ektp[i,j] = -tau[i,j]/(f*rho)
            ek[i,j]   = ektp[i,j]*dx[j]*1e-6         #v = taux/(rho*f*H), so tp = v*H*L = taux*L/(rho*f)
    
    ek85 = np.nansum(ek,axis=1)

    fs85v = np.zeros(t)
    AABW85v = np.zeros(t)
    lNADW85v = np.zeros(t)
    uNADW85v = np.zeros(t)
    iw85v = np.zeros(t)
    tr85v = np.zeros(t)
    tr85testv = np.zeros(t)
    ek85testv = np.zeros(t)   
    #Boundaries vary between years
    for i in range (t):
        #imax
        tprow85 = np.nansum(tp85[i,:,:],axis=1)
        mocbtv  = -1*np.nancumsum(tprow85[::-1])[::-1]
        ilevmin = np.nanargmin(np.abs(lev-200))         #Find index where lev = 200, so outlier where imax = 0 is solved
        ilevmax = np.nanargmin(np.abs(lev-2000))         #Find index where lev = 2000, so outlier where imax = -1 is solved
        
        if totmod[mod] == 'NESM3':                     #NESM3 has a peak in overturning at surface up till depths of ~700m
            ilevmin = np.nanargmin(np.abs(lev-800))
         
        if totmod[mod] == 'HadGEM3-GC31-MM':                     #HadGEM3-GC31-MM has a peak in overturning at surface up till depths of ~450m
            ilevmin = np.nanargmin(np.abs(lev-450))
            
        imax2   = np.where(mocbtv[ilevmin:ilevmax] == max(mocbtv[ilevmin:ilevmax]))
        imax1   = imax2[0]
        imax    = imax1[0] + ilevmin
        
        imaxtest2   = np.where(mocbtv == max(mocbtv))
        imaxtest1   = imaxtest2[0]
        imaxtest    = imaxtest1[0]
        
        if imax != imaxtest:
            print(str(totmod[mod]) + ' - ' + str(i) + ' - ' + str(lev[imax]))
        
        #Find FS boundary
        tpcol85 = np.nansum(tp85[i,0:imax,:],axis=0)
        mx     = np.nanargmin(np.abs(tpcol85 - max(tpcol85)))
        idx2   = np.where(tpcol85[mx::]<0)                     #Make sure idx > mx (max(tpcol85))
        idx1   = idx2[0]
        idx = idx1[0] + mx
        
        tprow_upmo85v    = np.nansum(tp85[i,:,idx::],axis=1)
        tprow_lowmo85v   = np.nansum(tp85[i,:,:],axis=1)

        #PT = 8
        PTrow = np.nanmean(T[i,:,:],axis=-1)
        PT8idx = np.nanargmin(np.abs(PTrow-8))
                
        if PT8idx > imax:                                       #Make sure PT8 !> imax
            PT8idx = imax

        #PT = 3
        PT3idx = np.nanargmin(np.abs(PTrow-3))
    
        #Aidx
        for m in range (1,len(tprow_lowmo85v)):                    #start at 1, otherwise the first and last point will be compared
            if tprow_lowmo85v[m]>0 and tprow_lowmo85v[m-1]<0:
                Aidx = m
                
        AABWcheck = tprow_lowmo85v[int(Aidx)::]                    #Make sure sum(AABW) is positive
        if sum(AABWcheck) < 0:
            Aidx = len(tprow_lowmo85v) - 1
            
        if PT3idx > Aidx:                                       #Make sure PT3 !> Aidx
            PT3idx = Aidx 
        
        tprow_fs85v    = np.nansum(tp85[i,0:imax,0:idx],axis=1)
        fs85v[i]       = np.nansum(tprow_fs85v)
        AABW85v[i]     = np.nansum(tprow_lowmo85v[int(Aidx)::])
        lNADW85v[i]    = np.nansum(tprow_lowmo85v[int(PT3idx):int(Aidx)])
        uNADW85v[i]    = np.nansum(tprow_lowmo85v[imax:int(PT3idx)])
        iw85v[i]       = np.nansum(tprow_upmo85v[PT8idx:imax])
        tr85v[i]       = np.nansum(tprow_upmo85v[0:PT8idx]) - ek85[i]         #tr = tp[upper] - ek
    
    #Write to file
    outfile = ('/home/users/jordi/components_tpBS/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid = Dataset(outfile,'w')
    ncid.createDimension('time', t)
    
    ncid.createVariable('fs85','f8',('time',))
    ncid.createVariable('AABW85','f8',('time',))
    ncid.createVariable('lNADW85','f8',('time',))
    ncid.createVariable('uNADW85','f8',('time',))
    ncid.createVariable('iw85','f8',('time',))
    ncid.createVariable('tr85','f8',('time',))
    ncid.createVariable('ek85','f8',('time',))
    
    ncid.variables['fs85'][:]       = fs85v
    ncid.variables['AABW85'][:]     = AABW85v
    ncid.variables['lNADW85'][:]    = lNADW85v
    ncid.variables['uNADW85'][:]    = uNADW85v
    ncid.variables['iw85'][:]       = iw85v
    ncid.variables['tr85'][:]       = tr85v
    ncid.variables['ek85'][:]       = ek85

    ncid.close()

computing file 1 of 19


/home/users/jmecking001/miniconda3/envs/jordi_2021/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: Warning: converting a masked element to nan.


CAMS-CSM1-0 - 553 - 1173.1890869140625
computing file 2 of 19
computing file 3 of 19
CESM2-WACCM - 889 - 527.7280078125
CESM2-WACCM - 890 - 482.73671875
CESM2-WACCM - 913 - 527.7280078125
computing file 4 of 19
CIESM - 457 - 579.3728637695312
CIESM - 554 - 482.7366943359375
CIESM - 565 - 579.3728637695312
CIESM - 586 - 482.7366943359375
CIESM - 725 - 443.377685546875
CIESM - 732 - 527.7279663085938
CIESM - 733 - 527.7279663085938
CIESM - 758 - 482.7366943359375
CIESM - 807 - 408.7846374511719
CIESM - 808 - 482.7366943359375
CIESM - 832 - 482.7366943359375
CIESM - 841 - 482.7366943359375
CIESM - 866 - 378.22760009765625
CIESM - 867 - 443.377685546875
CIESM - 868 - 443.377685546875
CIESM - 869 - 482.7366943359375
CIESM - 875 - 527.7279663085938
CIESM - 887 - 787.0025024414062
CIESM - 889 - 527.7279663085938
CIESM - 900 - 527.7279663085938
CIESM - 912 - 482.7366943359375
CIESM - 934 - 527.7279663085938
CIESM - 963 - 408.7846374511719
CIESM - 964 - 408.7846374511719
CIESM - 975 - 443.37768

/home/users/jmecking001/miniconda3/envs/jordi_2021/lib/python3.7/site-packages/ipykernel_launcher.py:81: RuntimeWarning: Mean of empty slice


computing file 6 of 19
computing file 7 of 19
CNRM-CM6-1 - 637 - 697.2586477464497
CNRM-CM6-1 - 697 - 773.3682589718316
CNRM-CM6-1 - 793 - 628.0259704911632
CNRM-CM6-1 - 818 - 773.3682589718316
CNRM-CM6-1 - 961 - 697.2586477464497
computing file 8 of 19
computing file 9 of 19
EC-Earth3 - 541 - 697.2586669921875
EC-Earth3 - 769 - 565.2922973632812
computing file 10 of 19
computing file 11 of 19
HadGEM3-GC31-LL - 301 - 856.678955078125
HadGEM3-GC31-LL - 445 - 773.3682861328125
HadGEM3-GC31-LL - 876 - 697.2586669921875
HadGEM3-GC31-LL - 877 - 628.0260009765625
HadGEM3-GC31-LL - 936 - 697.2586669921875
HadGEM3-GC31-LL - 938 - 565.2922973632812
HadGEM3-GC31-LL - 984 - 697.2586669921875
computing file 12 of 19


#### SSP585 - NADW recirculation 

In [18]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP   = 'ssp585'

for mod in range (len(totmod)):
    print(('computing file ' + str(mod+1) + ' of ' + str(len(totmod))))

    outfile = ('/home/users/jordi/section26data/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid    = Dataset(outfile,'r')
    lev     = ncid.variables['lev'][:]
    lat     = np.squeeze(ncid.variables['lat'][:,:])
    lon     = np.squeeze(ncid.variables['lon'][:,:])
    tp85    = ncid.variables['tp85_BS'][:,:,:]
    T       = ncid.variables['thetao'][:,:,0,:]
    SP      = ncid.variables['so'][:,:,0,:]
    tau     = ncid.variables['tauuo'][:,0,:]
    dx      = ncid.variables['dx'][0,:]
    ncid.close() 
    
    NADW85N    = np.zeros(time85)
    NADW85S    = np.zeros(time85)    
    #Boundaries vary between years
    for i in range (time85):
        #imax
        tprow85 = np.nansum(tp85[i,:,:],axis=1)
        mocbtv  = -1*np.nancumsum(tprow85[::-1])[::-1]
        ilevmin = np.nanargmin(np.abs(lev-200))         #Find index where lev = 200, so outlier where imax = 0 is solved
        ilevmax = np.nanargmin(np.abs(lev-2000))         #Find index where lev = 2000, so outlier where imax = -1 is solved
        imax2   = np.where(mocbtv[ilevmin:ilevmax] == max(mocbtv[ilevmin:ilevmax]))
        imax1   = imax2[0]
        imax    = imax1[0] + ilevmin                       #+ilevmin since imax[0] = mocbtv[ilev]
        
        imaxtest2   = np.where(mocbtv == max(mocbtv))
        imaxtest1   = imaxtest2[0]
        imaxtest    = imaxtest1[0]
        
        if imax != imaxtest:
            print(str(totmod[mod]) + ' - ' + str(i) + ' - ' + str(lev[imax]))
    
        #Aidx
        for m in range (1,len(tprow_lowmo85v)):                    #start at 1, otherwise the first and last point will be compared
            if tprow_lowmo85v[m]>0 and tprow_lowmo85v[m-1]<0:
                Aidx = m
                
        AABWcheck = tprow_lowmo85v[int(Aidx)::]                    #Make sure sum(AABW) is positive
        if sum(AABWcheck) < 0:
            Aidx = len(tprow_lowmo85v) - 1
                  
        #Recirculation boundaries
        NADWcol85  = np.nansum(tp85[i,imax:int(Aidx),:],axis=0)
        
        if np.nansum(NADWcol85) != 0:
            mxNADW   = np.nanargmin(NADWcol85)
            if mxNADW > len(lon)/2:                                         #Make sure min is on the western 'boundary'
                mxNADW = np.nanargmin(NADWcol85[0:int(len(lon)/2)])
            
            idx2NADW = np.where(NADWcol85[mxNADW::]>0)                      #Make sure idx > mx (max(tpcol85))
            idx1NADW = idx2NADW[0]
            if len(idx1NADW) != 0:
                idxNADW  = idx1NADW[0] + mxNADW
            else:
                idxNADW = -1

            NADW85N[i] = np.nansum(NADWcol85[idxNADW::])
            NADW85S[i] = np.nansum(NADWcol85[0:idxNADW])
        else:
            NADW85N[i] = 0
            NADW85S[i] = 0
                  
    #Write to file
    outfile = ('/home/users/jordi/components_tpBS_recir/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid = Dataset(outfile,'w')
    ncid.createDimension('time', time85)
    
    ncid.createVariable('NADW85N','f8',('time',))    
    ncid.createVariable('NADW85S','f8',('time',)) 

    ncid.variables['NADW85N'][:]    = NADW85N
    ncid.variables['NADW85S'][:]    = NADW85S

    ncid.close()

computing file 1 of 18


/home/users/jmecking001/miniconda3/envs/jordi_2021/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Warning: converting a masked element to nan.


CAMS-CSM1-0 - 553 - 1173.1890869140625
computing file 2 of 18
computing file 3 of 18
CESM2-WACCM - 889 - 527.7280078125
CESM2-WACCM - 890 - 482.73671875
CESM2-WACCM - 913 - 527.7280078125
computing file 4 of 18
CIESM - 457 - 579.3728637695312
CIESM - 554 - 482.7366943359375
CIESM - 565 - 579.3728637695312
CIESM - 586 - 482.7366943359375
CIESM - 725 - 443.377685546875
CIESM - 732 - 527.7279663085938
CIESM - 733 - 527.7279663085938
CIESM - 758 - 482.7366943359375
CIESM - 807 - 408.7846374511719
CIESM - 808 - 482.7366943359375
CIESM - 832 - 482.7366943359375
CIESM - 841 - 482.7366943359375
CIESM - 866 - 378.22760009765625
CIESM - 867 - 443.377685546875
CIESM - 868 - 443.377685546875
CIESM - 869 - 482.7366943359375
CIESM - 875 - 527.7279663085938
CIESM - 887 - 787.0025024414062
CIESM - 889 - 527.7279663085938
CIESM - 900 - 527.7279663085938
CIESM - 912 - 482.7366943359375
CIESM - 934 - 527.7279663085938
CIESM - 963 - 408.7846374511719
CIESM - 964 - 408.7846374511719
CIESM - 975 - 443.37768

/home/users/jmecking001/miniconda3/envs/jordi_2021/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: Mean of empty slice


computing file 6 of 18
computing file 7 of 18
CNRM-CM6-1 - 637 - 697.2586477464497
CNRM-CM6-1 - 697 - 773.3682589718316
CNRM-CM6-1 - 793 - 628.0259704911632
CNRM-CM6-1 - 818 - 773.3682589718316
CNRM-CM6-1 - 961 - 697.2586477464497
computing file 8 of 18
computing file 9 of 18
EC-Earth3 - 541 - 697.2586669921875
EC-Earth3 - 769 - 565.2922973632812
computing file 10 of 18
computing file 11 of 18
HadGEM3-GC31-LL - 301 - 856.678955078125
HadGEM3-GC31-LL - 445 - 773.3682861328125
HadGEM3-GC31-LL - 876 - 697.2586669921875
HadGEM3-GC31-LL - 877 - 628.0260009765625
HadGEM3-GC31-LL - 936 - 697.2586669921875
HadGEM3-GC31-LL - 938 - 565.2922973632812
HadGEM3-GC31-LL - 984 - 697.2586669921875
computing file 12 of 18
HadGEM3-GC31-MM - 73 - 697.2586669921875
HadGEM3-GC31-MM - 74 - 856.678955078125
HadGEM3-GC31-MM - 205 - 697.2586669921875
HadGEM3-GC31-MM - 206 - 697.2586669921875
HadGEM3-GC31-MM - 614 - 773.3682861328125
HadGEM3-GC31-MM - 708 - 697.2586669921875
HadGEM3-GC31-MM - 709 - 856.678955078

#### Historical since 1950 (for lagged regression)

In [4]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP   = 'historical'

for mod in range (len(totmod)):
    print(('computing file ' + str(mod+1) + ' of ' + str(len(totmod))))

    outfile = ('/home/users/jordi/section26data/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid    = Dataset(outfile,'r')
    lev     = ncid.variables['lev'][:]
    lat     = np.squeeze(ncid.variables['lat'][:,:])
    lon     = np.squeeze(ncid.variables['lon'][:,:])
    T       = ncid.variables['thetao'][:,:,0,:]
    SP      = ncid.variables['so'][:,:,0,:]
    tau     = ncid.variables['tauuo'][:,0,:]
    dx      = ncid.variables['dx'][0,:]
    ncid.close()

    outfile = ('/home/users/jordi/tp1950_BS.nc')    
    ncid = Dataset(outfile,'r')    
    tp10 = ncid.variables[totmod[mod]][:,:,:]
    ncid.close()    
    
    #Ekman calculation
    f = 2*7.2921e-5*np.sin(lat[0]*np.pi/180)
    rho = 1026

    ektp = np.zeros(np.shape(tau))
    ek = np.zeros(np.shape(tau))
    for i in range (len(tau[:,0])):
        for j in range (len(lon)):
            ektp[i,j] = -tau[i,j]/(f*rho)
            ek[i,j]   = ektp[i,j]*dx[j]*1e-6         #v = taux/(rho*f*H), so tp = v*H*L = taux*L/(rho*f)
    
    ektot = np.nansum(ek,axis=1)
    
    time10 = 780

    fs10v = np.zeros(time10)
    AABW10v = np.zeros(time10)
    lNADW10v = np.zeros(time10)
    uNADW10v = np.zeros(time10)
    iw10v = np.zeros(time10)
    tr10v = np.zeros(time10)
    ek10 = ektot[-780::]
    tr10testv = np.zeros(time10)
    ek10testv = np.zeros(time10)   
    #Boundaries vary between years
    for i in range (time10):
        #imax
        tprow10 = np.nansum(tp10[i,:,:],axis=1)
        mocbtv  = -1*np.nancumsum(tprow10[::-1])[::-1]
        ilevmin = np.nanargmin(np.abs(lev-200))         #Find index where lev = 200, so outlier where imax = 0 is solved
        ilevmax = np.nanargmin(np.abs(lev-2000))         #Find index where lev = 2000, so outlier where imax = -1 is solved
        imax2   = np.where(mocbtv[ilevmin:ilevmax] == max(mocbtv[ilevmin:ilevmax]))
        imax1   = imax2[0]
        imax    = imax1[0] + ilevmin                       #+ilevmin since imax[0] = mocbtv[ilev]
        
        imaxtest2   = np.where(mocbtv == max(mocbtv))
        imaxtest1   = imaxtest2[0]
        imaxtest    = imaxtest1[0]
        
        if imax != imaxtest:
            print(str(totmod[mod]) + ' - ' + str(i) + ' - ' + str(lev[imax]))
            
        #Find FS boundary
        tpcol10 = np.nansum(tp10[i,0:imax,:],axis=0)
        mx     = np.nanargmin(np.abs(tpcol10 - max(tpcol10)))
        idx2   = np.where(tpcol10[mx::]<0)                      #Make sure idx > mx (max(tpcol10))
        idx1   = idx2[0]
        idx = idx1[0] + mx
        
        tprow_upmo10v    = np.nansum(tp10[i,:,idx::],axis=1)
        tprow_lowmo10v   = np.nansum(tp10[i,:,:],axis=1)
        
        #PT = 8
        T10  = T[-780::,:,:]
        PTrow = np.nanmean(T10[i,:,:],axis=-1)
        PT8idx = np.nanargmin(np.abs(PTrow-8))
              
        if PT8idx > imax:                                       #Make sure PT8 !> imax
            PT8idx = imax

        #PT = 3
        PT3idx = np.nanargmin(np.abs(PTrow-3))
    
        #Aidx
        for m in range (1,len(tprow_lowmo10v)):                    #start at 1, otherwise the first and last point will be compared
            if tprow_lowmo10v[m]>0 and tprow_lowmo10v[m-1]<0:
                Aidx = m
                
        AABWcheck = tprow_lowmo10v[int(Aidx)::]                    #Make sure sum(AABW) is positive
        if sum(AABWcheck) < 0:
            Aidx = len(tprow_lowmo10v) - 1
            
        if PT3idx > Aidx:                                       #Make sure PT3 !> Aidx
            PT3idx = Aidx 
        
        tprow_fs10v    = np.nansum(tp10[i,0:imax,0:idx],axis=1)
        fs10v[i]       = np.nansum(tprow_fs10v)
        AABW10v[i]     = np.nansum(tprow_lowmo10v[int(Aidx)::])
        lNADW10v[i]    = np.nansum(tprow_lowmo10v[int(PT3idx):int(Aidx)])
        uNADW10v[i]    = np.nansum(tprow_lowmo10v[imax:int(PT3idx)])
        iw10v[i]       = np.nansum(tprow_upmo10v[PT8idx:imax])
        tr10v[i]       = np.nansum(tprow_upmo10v[0:PT8idx]) - ek10[i]         #tr = tp[0-800m] - ek
    
    #Write to file
    outfile = ('/home/users/jordi/components_tp1950/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid = Dataset(outfile,'w')
    ncid.createDimension('time', time10)
    
    ncid.createVariable('fs10','f8',('time',))
    ncid.createVariable('AABW10','f8',('time',))
    ncid.createVariable('lNADW10','f8',('time',))
    ncid.createVariable('uNADW10','f8',('time',))
    ncid.createVariable('iw10','f8',('time',))
    ncid.createVariable('tr10','f8',('time',))
    ncid.createVariable('ek10','f8',('time',))
    
    ncid.variables['fs10'][:]       = fs10v
    ncid.variables['AABW10'][:]     = AABW10v
    ncid.variables['lNADW10'][:]    = lNADW10v
    ncid.variables['uNADW10'][:]    = uNADW10v
    ncid.variables['iw10'][:]       = iw10v
    ncid.variables['tr10'][:]       = tr10v
    ncid.variables['ek10'][:]       = ek10

    ncid.close()

computing file 1 of 20


/home/users/jmecking001/miniconda3/envs/jordi_2021/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Warning: converting a masked element to nan.


computing file 2 of 20
computing file 3 of 20
computing file 4 of 20
computing file 5 of 20


/home/users/jmecking001/miniconda3/envs/jordi_2021/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: Mean of empty slice


computing file 6 of 20
computing file 7 of 20
computing file 8 of 20
computing file 9 of 20
computing file 10 of 20
computing file 11 of 20
computing file 12 of 20
computing file 13 of 20
computing file 14 of 20
computing file 15 of 20
computing file 16 of 20
computing file 17 of 20
computing file 18 of 20
computing file 19 of 20
NESM3 - 145 - 1136.9219970703125
NESM3 - 146 - 989.2288818359375
NESM3 - 242 - 1136.9219970703125
NESM3 - 289 - 1136.9219970703125
NESM3 - 326 - 1136.9219970703125
computing file 20 of 20
